# Prerequisite

Following packages must be installed to run codes below

```bash
$ pip3 install openpyxl 
```


In [1]:
%load_ext autoreload
%autoreload 2

# Add project path

In [2]:
import os
import sys
from pathlib import Path

PROJECT_PATH = Path().resolve().parent.parent
SRC_PATH = PROJECT_PATH / 'src'

sys.path.append(str(SRC_PATH))

In [3]:
exp_name = 'feature_seleciton_v1'

# Load data

In [4]:
from data_loader import SectorDataLoader

data_loader = SectorDataLoader()
data_finance = (
    data_loader
    .load_dataset()
    .pipe(data_loader.add_beaver_indicator)
    .pipe(data_loader.add_label)
)
data_finance.head()

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_TYP,CMP_SCL,PBCO_GB,...,STAT_OCR_DATE_y,Closed_Year,유동자산/부채총계,당기순이익(손실)/자산총계,부채총계/자산총계,순운전자본/자산총계,유동부채/유동자산,Years_From_Closed_Year_To_FS,Closed_In_1Yr,Closed_In_2Yrs
117,1018135422,(주),한국선박기술,NaN,NaN,Korea Marine Time Service,M,NaN,2,2,...,20200930.0,2020.0,1.031451,0.038773,0.578094,0.543045,0.089273,2.0,0,1
219,1018154206,(주),드림미즈,NaN,NaN,"dreammiz Co., Ltd.",M,NaN,2,2,...,20201231.0,2020.0,1.220040,0.001177,0.662356,0.608449,0.247063,2.0,0,1
279,1018163684,NaN,디에프에스서울,NaN,(주),DFS Seoul Ltd.,M,NaN,2,2,...,20190917.0,2019.0,11.012914,0.046666,0.090794,0.909114,0.090802,1.0,1,1
339,1018178760,(주),대교디앤씨,NaN,NaN,"DAEGYO D & C CO.,LTD.",M,NaN,2,2,...,20191010.0,2019.0,0.916364,-0.026128,1.091269,0.097514,0.902486,1.0,1,1
406,1018194173,NaN,에코에너지,NaN,(주),"Eco Energy Co.,Ltd.",M,NaN,2,2,...,20200831.0,2020.0,1.183355,-0.062095,0.828142,0.194625,0.801400,2.0,0,1


In [5]:
data = data_finance

# Baseline classifier using Beaver's features

In [6]:
from project_paths import DATA_PATH

sectors = [
    '제조업(10~34)',
    '부동산업(68)',
    '도매 및 소매업(45~47)',
    '숙박 및 음식점업(55~56)',
    '건설업(41~42)'
]

beaver_features = [
    '유동자산/부채총계',
    '당기순이익(손실)/자산총계',
    '부채총계/자산총계',
    '순운전자본/자산총계',
    '유동부채/유동자산'
]

extended_financial_features = [
    '유동자산',
    '매출채권',
    '비유동자산',
    '유형자산',
    '자산총계',
    '유동부채',
    '비유동부채',
    '부  채  총  계',
    '자본금',
    '이익잉여금(결손금）',
    '자본총계',
    '매출액',
    '판매비와관리비',
    '영업이익（손실）',
    '법인세비용차감전순손익',
    '법인세비용',
    '당기순이익(손실)',
    '기업순이익률(%)',
    '유보액/총자산(%)',
    '유보액/납입자본(%)',
    '매출액총이익률(%)',
    '매출액영업이익률(%)',
    '매출액순이익률(%)',
    '수지비율(%)',
    '경상수지비율',
    '영업비율(%)',
    '금융비용대매출액비율(%',
    '금융비용대부채비율(%)',
    '금융비용대총비용비율(%',
    '부채비율(%)',
    '차입금의존도(%)',
    '자기자본비율(%)',
    '순운전자본비율(%)',
    '유동부채비율(%)',
    '비유동부채비율(%)',
    '부채총계대 매출액(%)',
    '총자본회전율(회)',
    '재고자산회전율(회)',
    '매출채권회전율(회)',
    '매입채무회전율(회)',
    '미수금',
    '매출원가',
    '무형자산',
    '재고자산',
]

years_to_close = [
    'Closed_In_1Yr',
    'Closed_In_2Yrs',
]

x_data_year = list(range(2018, 2021))

EXP_RESULT_PATH = DATA_PATH / 'experiment_result' / exp_name

if not os.path.exists(EXP_RESULT_PATH) : 
    os.mkdir(EXP_RESULT_PATH)

In [7]:
import numpy as np
import pandas as pd
from pycaret.classification import ClassificationExperiment
from tqdm import tqdm

In [8]:
exp_result = None

for target_years_to_close in tqdm(years_to_close) : 
    for x_data_target_year in tqdm(x_data_year, leave=True) : 
        for sector in sectors : 

            filename = EXP_RESULT_PATH / f'Beaver_baseline_{sector}_{target_years_to_close}_{x_data_target_year}.csv'

            if not os.path.exists(filename) : 

                data_to_train = (
                    data
                    .loc[~data[beaver_features[0]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[1]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[2]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[3]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[~data[beaver_features[4]].isin([np.nan, np.inf, -np.inf]), :]
                    .loc[lambda x : pd.to_datetime(x['결산년월'], format='%Y%m%d').dt.year==x_data_target_year]
                    .loc[lambda x : x['대분류']==sector]
                )

                exp = ClassificationExperiment()

                exp.setup(
                    (
                        data_to_train
                        .loc[:, beaver_features+extended_financial_features+[target_years_to_close]]
                        .reset_index(drop=True)
                    ),
                    target=target_years_to_close
                )

                models = exp.compare_models()

                result = (
                    exp.pull()
                    .assign(target=target_years_to_close)
                    .assign(x_data_yaer=x_data_target_year)
                    .assign(대분류=sector)
                    .reset_index(drop=False)
                    .rename(columns={'index':'model_name'})
                )
                result.to_csv(filename)

            else : 
                result = pd.read_csv(filename)

            if exp_result is None : 
                exp_result = result
            else : 
                exp_result = pd.concat([exp_result, result], axis=0)

  0%|                                                                                    | 0/3 [00:00<?, ?it/s]

,Description,Value
0,Session id,5814
1,Target,Closed_In_1Yr
2,Target type,Binary
3,Original data shape,"(1800, 50)"
4,Transformed data shape,"(1800, 50)"
5,Transformed train set shape,"(1259, 50)"
6,Transformed test set shape,"(541, 50)"
7,Numeric features,49
8,Rows with missing values,84.8%
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8769,0.6900,0.1349,0.6762,0.2162,0.1810,0.2532,0.1190
et,Extra Trees Classifier,0.8745,0.6817,0.1349,0.6492,0.2148,0.1766,0.2444,0.0830
lightgbm,Light Gradient Boosting Machine,0.8713,0.6923,0.1221,0.5667,0.1974,0.1568,0.2146,0.1070
lr,Logistic Regression,0.8697,0.5062,0.0000,0.0000,0.0000,0.0000,0.0000,0.2280
qda,Quadratic Discriminant Analysis,0.8697,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0120
dummy,Dummy Classifier,0.8697,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0090
gbc,Gradient Boosting Classifier,0.8681,0.6823,0.1221,0.5142,0.1884,0.1453,0.1922,0.2290
ridge,Ridge Classifier,0.8626,0.0000,0.0419,0.3483,0.0717,0.0429,0.0767,0.0070
lda,Linear Discriminant Analysis,0.8554,0.5946,0.0426,0.1783,0.0671,0.0282,0.0332,0.0080
ada,Ada Boost Classifier,0.8546,0.6869,0.0735,0.2804,0.1102,0.0623,0.0796,0.0670


,Description,Value
0,Session id,4026
1,Target,Closed_In_1Yr
2,Target type,Binary
3,Original data shape,"(323, 50)"
4,Transformed data shape,"(323, 50)"
5,Transformed train set shape,"(226, 50)"
6,Transformed test set shape,"(97, 50)"
7,Numeric features,49
8,Rows with missing values,98.5%
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8227,0.6798,0.3000,0.6500,0.3988,0.3164,0.3521,0.0460
rf,Random Forest Classifier,0.8097,0.6751,0.2350,0.6000,0.3254,0.2477,0.2914,0.0670
lightgbm,Light Gradient Boosting Machine,0.7964,0.6601,0.2550,0.5667,0.3421,0.2396,0.2719,0.0120
dummy,Dummy Classifier,0.7923,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0070
lda,Linear Discriminant Analysis,0.7528,0.6081,0.2200,0.2667,0.2305,0.1113,0.1106,0.0060
gbc,Gradient Boosting Classifier,0.7476,0.6690,0.2700,0.3467,0.3002,0.1516,0.1539,0.0390
ada,Ada Boost Classifier,0.7437,0.6128,0.2200,0.3019,0.2421,0.1071,0.1088,0.0240
lr,Logistic Regression,0.7344,0.5577,0.2400,0.3126,0.2650,0.1140,0.1143,0.0590
knn,K Neighbors Classifier,0.7215,0.5556,0.0800,0.2500,0.1111,-0.0291,-0.0131,0.0150
ridge,Ridge Classifier,0.6988,0.0000,0.4100,0.3359,0.3440,0.1627,0.1762,0.0050


...

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0380
nb,Naive Bayes,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0080
dt,Decision Tree Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0090
ridge,Ridge Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0060
rf,Random Forest Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0460
ada,Ada Boost Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0080
lda,Linear Discriminant Analysis,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0080
et,Extra Trees Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0370
lightgbm,Light Gradient Boosting Machine,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0080
dummy,Dummy Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0070


,Description,Value
0,Session id,8718
1,Target,Closed_In_2Yrs
2,Target type,Multiclass
3,Target mapping,1: 0
4,Original data shape,"(11, 50)"
5,Transformed data shape,"(11, 50)"
6,Transformed train set shape,"(7, 50)"
7,Transformed test set shape,"(4, 50)"
8,Numeric features,49
9,Rows with missing values,81.8%


,Description,Value
0,Session id,695
1,Target,Closed_In_2Yrs
2,Target type,Multiclass
3,Target mapping,1: 0
4,Original data shape,"(100, 50)"
5,Transformed data shape,"(100, 50)"
6,Transformed train set shape,"(69, 50)"
7,Transformed test set shape,"(31, 50)"
8,Numeric features,49
9,Rows with missing values,95.0%


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0390
nb,Naive Bayes,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0080
dt,Decision Tree Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0080
ridge,Ridge Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0060
rf,Random Forest Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0480
ada,Ada Boost Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0070
lda,Linear Discriminant Analysis,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0080
et,Extra Trees Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0380
lightgbm,Light Gradient Boosting Machine,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0080
dummy,Dummy Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,nan,0.0000,0.0070



100%|████████████████████████████████████████████████████████████████████████████| 2/2 [02:54<00:00, 87.25s/it]


In [9]:
aggregated_result_path = EXP_RESULT_PATH / 'result.csv'

if not os.path.exists(aggregated_result_path) :
    exp_result.to_csv(aggregated_result_path)
    
exp_result

,model_name,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec),target,x_data_yaer,대분류
0,rf,Random Forest Classifier,0.8769,0.6900,0.1349,0.6762,0.2162,0.1810,0.2532,0.119,Closed_In_1Yr,2018,제조업(10~34)
1,et,Extra Trees Classifier,0.8745,0.6817,0.1349,0.6492,0.2148,0.1766,0.2444,0.083,Closed_In_1Yr,2018,제조업(10~34)
2,lightgbm,Light Gradient Boosting Machine,0.8713,0.6923,0.1221,0.5667,0.1974,0.1568,0.2146,0.107,Closed_In_1Yr,2018,제조업(10~34)
3,lr,Logistic Regression,0.8697,0.5062,0.0000,0.0000,0.0000,0.0000,0.0000,0.228,Closed_In_1Yr,2018,제조업(10~34)
4,qda,Quadratic Discriminant Analysis,0.8697,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.012,Closed_In_1Yr,2018,제조업(10~34)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,ada,Ada Boost Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,NaN,0.0000,0.007,Closed_In_2Yrs,2020,건설업(41~42)
6,lda,Linear Discriminant Analysis,1.0000,0.0000,1.0000,1.0000,1.0000,NaN,0.0000,0.008,Closed_In_2Yrs,2020,건설업(41~42)
7,et,Extra Trees Classifier,1.0000,0.0000,1.0000,1.0000,1.0000,NaN,0.0000,0.038,Closed_In_2Yrs,2020,건설업(41~42)
8,lightgbm,Light Gradient Boosting Machine,1.0000,0.0000,1.0000,1.0000,1.0000,NaN,0.0000,0.008,Closed_In_2Yrs,2020,건설업(41~42)
